## Resume

In [1]:
import csv
from time import sleep
from getpass import getpass

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [2]:
driver = Chrome()
driver.get('https://www.twitter.com/login')
driver.maximize_window()

In [3]:
username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('cgomezloaiza@gmail.com')
my_password = getpass()


password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(1)

········


In [4]:
profile = 'BillGates'

In [5]:
driver.get('https://www.twitter.com/'+ profile)

In [6]:
cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')

In [7]:
card = cards[0]

In [8]:
card.find_element_by_xpath('./div[2]/div[1]//span').text

'Gates Foundation'

In [9]:
def get_tweet_data(card):
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(),"@")]').text
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element_by_xpath('./div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('./div[2]/div[2]/div[2]').text
    text = comment + responding
    
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    
    return tweet

In [10]:
get_tweet_data(card)

('Gates Foundation',
 '@gatesfoundation',
 '2021-05-06T23:39:37.000Z',
 'As our CEO \n@MSuzman\n says, no barriers should stand in the way of equitable access to vaccines. We are supportive of a narrow intellectual property waiver for COVID-19 vaccines during the pandemic.Statement from Gates Foundation CEO Mark Suzman: No barriers should stand in the way of equitable...\nThe foundation is supportive of a narrow IP waiver during the pandemic.\ngatesfoundation.org',
 '2,3 mil',
 '651',
 '2,8 mil')

In [11]:
tweet_data = []

for card in cards:
    data = get_tweet_data(card)
    if data:
        tweet_data.append(data)

In [12]:
card.find_element_by_xpath('.//div[@data-testid="retweet"]').text

'784'

In [13]:
tweet_data

[('Gates Foundation',
  '@gatesfoundation',
  '2021-05-06T23:39:37.000Z',
  'As our CEO \n@MSuzman\n says, no barriers should stand in the way of equitable access to vaccines. We are supportive of a narrow intellectual property waiver for COVID-19 vaccines during the pandemic.Statement from Gates Foundation CEO Mark Suzman: No barriers should stand in the way of equitable...\nThe foundation is supportive of a narrow IP waiver during the pandemic.\ngatesfoundation.org',
  '2,3 mil',
  '651',
  '2,8 mil'),
 ('Bill Gates',
  '@BillGates',
  '2021-05-03T20:30:19.000Z',
  '',
  '',
  '98,3 mil',
  '218,5 mil'),
 ('World Health Organization (WHO)',
  '@WHO',
  '2021-04-23T15:14:59.000Z',
  'The \n@ACTAccelerator\n, an unprecedented global alliance to develop & deliver the tests, treatments & vaccines the  needs to fight #COVID19, marks its first anniversary today with a special report on its progress & aims for 2021:\nhttps://bit.ly/3sFJjLLDe \nWorld Health Organization (WHO)',
  '1,7 mil',


In [14]:
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == 4:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 4:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = 4
            break

In [15]:
data[0]

('Gates Foundation',
 '@gatesfoundation',
 '2021-05-06T23:39:37.000Z',
 'As our CEO \n@MSuzman\n says, no barriers should stand in the way of equitable access to vaccines. We are supportive of a narrow intellectual property waiver for COVID-19 vaccines during the pandemic.Statement from Gates Foundation CEO Mark Suzman: No barriers should stand in the way of equitable...\nThe foundation is supportive of a narrow IP waiver during the pandemic.\ngatesfoundation.org',
 '2,3 mil',
 '651',
 '2,8 mil')

In [16]:
with open('tweets_table.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv("tweets_table.csv")

In [19]:
df.head(100)

,UserName,Handle,Timestamp,Text,Comments,Likes,Retweets
0,Gates Foundation,@gatesfoundation,2021-05-06T23:39:37.000Z,"As our CEO \n@MSuzman\n says, no barriers shou...","2,3 mil",651,"2,8 mil"
1,Bill Gates,@BillGates,2021-05-03T20:30:19.000Z,NaN,NaN,"98,3 mil","218,5 mil"
2,World Health Organization (WHO),@WHO,2021-04-23T15:14:59.000Z,"The \n@ACTAccelerator\n, an unprecedented glob...","1,7 mil",784,"3,5 mil"
3,Bill Gates,@BillGates,2021-04-06T15:42:53.000Z,The amount of cement China has consumed is a s...,NaN,787,"4,2 mil"
4,Bill Gates,@BillGates,2021-04-05T15:20:52.000Z,Who installed the most concrete?\n\nCheck your...,1,628,"3,6 mil"
5,Bill Gates,@BillGates,2021-04-05T15:25:24.000Z,Cement is responsible for 6% of carbon emissio...,NaN,512,"3,3 mil"
6,Bill Gates,@BillGates,2021-04-02T17:03:10.000Z,The best way to prevent new variants from emer...,NaN,"1,7 mil","6,1 mil"
7,Bill Gates,@BillGates,2021-04-01T20:38:26.000Z,It's encouraging to see innovation and clean e...,NaN,666,"4,3 mil"
8,Bill Gates,@BillGates,2021-03-31T18:47:05.000Z,Here are five things you should know if you wa...,NaN,702,"2,9 mil"
9,IFPRI,@IFPRI,2021-03-11T16:30:10.000Z,"From #climateadaptation to #genderequality, d...",260,120,639
